<a href="https://colab.research.google.com/github/Kei-Sanada/Numerai/blob/master/238_Modeling_20201115_2300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Round236にはこちらを提出

In [1]:
# install dependencies
!pip install pandas sklearn numerapi

In [2]:
!free -h 

              total        used        free      shared  buff/cache   available
Mem:            12G        559M         11G        956K        496M         11G
Swap:            0B          0B          0B


In [3]:
import pandas as pd
import numerapi

In [4]:
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
#training_data.head()

In [5]:
training_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,0.25,0.75,0.75,0.25,0.25,1.00,0.75,0.5,1.0,0.5,0.00,0.50,0.50,0.00,0.0,0.00,1.00,0.25,0.00,0.50,0.25,0.75,0.50,1.00,0.75,0.75,0.5,0.50,0.75,0.5,...,0.75,0.75,0.75,0.50,1.00,1.00,0.50,0.75,0.5,0.25,0.25,0.75,0.50,1.00,0.5,0.75,0.75,0.25,0.50,1.00,0.75,0.50,0.50,1.00,0.25,0.5,0.50,0.50,0.75,1.00,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.50
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,0.25,0.50,0.50,0.00,0.50,0.00,0.50,0.5,0.5,0.5,0.50,0.25,0.25,0.50,0.0,1.00,0.50,0.50,0.50,0.75,0.50,0.50,0.75,0.25,0.50,0.75,0.5,0.25,0.75,0.5,...,0.25,0.25,0.25,1.00,1.00,0.50,0.50,0.50,0.0,0.25,1.00,0.50,1.00,1.00,0.5,0.50,0.50,1.00,0.25,0.75,1.00,0.25,0.25,1.00,0.50,0.5,0.50,0.75,0.75,0.75,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,0.25,0.00,0.25,0.50,1.00,0.50,0.75,0.5,0.5,1.0,0.50,0.50,0.50,0.25,0.0,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,0.50,0.75,0.25,0.5,0.50,0.75,0.5,...,0.25,1.00,1.00,1.00,0.50,1.00,1.00,1.00,0.5,1.00,0.00,1.00,1.00,0.50,1.0,0.75,1.00,0.00,0.50,0.75,0.00,1.00,0.50,0.50,0.75,1.0,0.75,1.00,0.25,0.50,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.25
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,0.75,0.25,0.50,0.50,0.50,0.75,0.50,1.0,0.5,0.5,0.00,1.00,0.00,0.75,0.0,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.75,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,1.00,0.75,0.75,1.00,0.75,0.75,0.75,1.0,0.75,1.00,0.75,1.00,0.75,1.0,0.00,0.50,0.75,1.00,0.75,1.00,0.75,1.00,1.00,0.00,0.5,0.75,0.75,1.00,0.75,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.25
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.0,0.5,0.5,0.25,0.00,0.50,0.00,0.5,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,0.25,0.75,1.00,0.75,0.00,0.00,0.75,0.5,1.00,0.50,0.75,0.25,0.50,0.0,0.50,0.50,0.50,0.75,0.75,0.50,0.75,0.25,0.75,0.50,0.5,0.25,0.25,0.75,0.50,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75


In [6]:
tournament_data  = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
#tournament_data.head()

In [7]:
#tournament_data['target_kazutsugi'].unique()

In [8]:
# import dependencies
import sklearn.linear_model

In [9]:
# find only the feature columns
feature_cols = [f for f in training_data.columns if "feature" in f]

# select those columns out of the training dataset
training_features = training_data[feature_cols]

In [10]:
# create a model and fit the training data (~30 sec to run)
model = sklearn.linear_model.LinearRegression()
#model.fit(training_features, training_data.target_kazutsugi)
model.fit(training_features, training_data.target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
# select the feature columns from the tournament data
live_features = tournament_data[feature_cols]

In [12]:
# predict the target on the live features
predictions_sklearn_linear_model = model.predict(live_features)

正規化 https://forum.numer.ai/t/model-diagnostics-feature-exposure/899

In [13]:
feature_names_dexterity = [f for f in training_data.columns if "feature_dexterity" in f]
feature_names_intelligence = [f for f in training_data.columns if "feature_intelligence" in f]


In [14]:
feature_names_dexterity
feature_names_intelligence

['feature_intelligence1',
 'feature_intelligence2',
 'feature_intelligence3',
 'feature_intelligence4',
 'feature_intelligence5',
 'feature_intelligence6',
 'feature_intelligence7',
 'feature_intelligence8',
 'feature_intelligence9',
 'feature_intelligence10',
 'feature_intelligence11',
 'feature_intelligence12']

In [15]:
 #training_data["target_kazutsugi"]

In [16]:
 from xgboost import XGBRegressor

In [17]:
# train a model to make predictions on tournament data
#feature_names_dexterity
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       #max_depth=3,
       max_depth=2, #2020/11/15 Mod by Kei
       #min_child_weight=1,
       min_child_weight=3, #2020/11/15 Mod by Kei
       missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity =1,
       alpha=0.1) #2020/11/01 alpha=0.1　重みに関するL1正則化項高次元の場合に用いるらしい。
#model.fit(training_data[feature_names_dexterity], training_data["target_kazutsugi"])
model.fit(training_data[feature_names_dexterity], training_data["target"])
# submit predictions to numer.ai
predictions_dexterity = model.predict(tournament_data[feature_names_dexterity])

In [18]:
# train a model to make predictions on tournament data
#feature_names_intelligence
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       #max_depth=3,
       max_depth=2, #2020/11/15 Mod by Kei
       #min_child_weight=1,
       min_child_weight=3, #2020/11/15 Mod by Kei
       missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity =1,
       alpha=0.1) #2020/11/01 alpha=0.1　重みに関するL1正則化項高次元の場合に用いるらしい。
#model.fit(training_data[feature_names_dexterity], training_data["target_kazutsugi"])
model.fit(training_data[feature_names_intelligence], training_data["target"])

# submit predictions to numer.ai
predictions_intelligence = model.predict(tournament_data[feature_names_intelligence])

In [19]:
predictions = (predictions_sklearn_linear_model + predictions_dexterity + predictions_intelligence) / 3

In [20]:
#tournament_data["id"]

In [21]:
# predictions must have an `id` column and a `prediction_kazutsugi` column
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions
predictions_df.head()

,id,prediction_kazutsugi
0,n0003aa52cab36c2,0.490069
1,n000920ed083903f,0.496387
2,n0038e640522c4a6,0.510109
3,n004ac94a87dc54b,0.499386
4,n0052fe97ea0c05f,0.499819


In [28]:
import numpy as np
ranked_prediction = training_data["target"].rank(pct=True, method="first")
correlation = np.corrcoef(training_data["target"], ranked_prediction)[0, 1]

In [23]:
ranked_prediction

0         0.249239
1         0.049854
2         0.049856
3         0.049858
4         0.750779
            ...   
501803    0.750773
501804    0.950146
501805    0.249237
501806    0.750775
501807    0.750777
Name: target, Length: 501808, dtype: float64

In [24]:
correlation

0.9095228541212714

In [25]:
#training_data["target_kazutsugi"]

In [26]:
# Get your API keys and model_id from https://numer.ai/submit
public_id = "CYATEL5QQBU6APNFLCV7HEE7PV6SC7V6"
secret_key = "Y22BTSUGU4JEFGQB3RZNEESSULKA3HQJPAW3KI6BIXH2AMNMCTC44IFWTOQIO2UW"
model_id = "3c77ba09-cfa2-4b18-b789-918340c84c82"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [27]:
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)

2020-11-15 14:31:14,893 INFO numerapi.base_api: uploading predictions...


sklearn_linear_modelと　XGBRegressor

DIAGNOSTICS
Performance
Validation Sharpe
0.4237
Validation Mean
0.0141
Feature Neutral Mean
-0.001
Risk
Validation SD
0.0332
Feature Exposure
0.4039
Max Drawdown
-0.1324
MMC
Corr + MMC Sharpe
0.2281
MMC Mean
-0.0033
Corr With Example Preds
0.6934